In [1]:
import strawberryfields as sf
import numpy as np
from braket.strawberryfields_plugin import BraketEngine

eng = BraketEngine("arn:aws:braket:us-east-1::device/qpu/xanadu/Borealis")
device = eng.device

2023-03-27 14:58:42.478308: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
device.certificate["squeezing_parameters_mean"]

{'low': 0.714, 'high': 1.178, 'medium': 0.988}

In [3]:
device.certificate

{'finished_at': '2023-03-27T13:51:27.619274+00:00',
 'target': 'borealis',
 'loop_phases': [-1.698, 1.005, 0.585],
 'schmidt_number': 1.137,
 'common_efficiency': 0.368,
 'loop_efficiencies': [0.876, 0.89, 0.728],
 'squeezing_parameters_mean': {'low': 0.714, 'high': 1.178, 'medium': 0.988},
 'relative_channel_efficiencies': [0.933,
  0.937,
  0.917,
  0.999,
  0.972,
  0.921,
  0.9,
  0.967,
  0.959,
  0.959,
  0.972,
  1.0,
  0.958,
  0.975,
  0.955,
  0.899]}

In [4]:
from strawberryfields.tdm import borealis_gbs, get_mode_indices

gate_args_list = borealis_gbs(device, modes=216, squeezing="high")
delays = [1, 6, 36]
n, N = get_mode_indices(delays)

2023-03-27 15:01:32,155 - WARNING - 127/259 arguments of phase gate 1 have been shifted by pi in order to be compatible with the phase modulators.
2023-03-27 15:01:32,159 - WARNING - 127/259 arguments of phase gate 2 have been shifted by pi in order to be compatible with the phase modulators.


In [5]:
from strawberryfields.ops import Sgate, Rgate, BSgate, MeasureFock

prog = sf.TDMProgram(N)

with prog.context(*gate_args_list) as (p, q):
    Sgate(p[0]) | q[n[0]]
    for i in range(len(delays)):
        Rgate(p[2 * i + 1]) | q[n[i]]
        BSgate(p[2 * i + 2], np.pi / 2) | (q[n[i + 1]], q[n[i]])
    MeasureFock() | q[0]

In [6]:
shots = 1000
results = eng.run(prog, shots=shots, crop=True)
print(results.samples)

[[[0 2 0 ... 1 2 0]]

 [[0 0 1 ... 0 0 0]]

 [[0 0 0 ... 0 0 0]]

 ...

 [[0 0 0 ... 0 0 0]]

 [[0 0 0 ... 0 0 0]]

 [[1 0 1 ... 0 0 0]]]
